# DATASET e DATALOADER
Il codice per elaborare campioni di dati può diventare disordinato e difficile da mantenere; idealmente vogliamo che il nostro codice del dataset sia diviso dal codice per l'addestramento del modello per una migliore leggibilità e modularità. PyTorch fornisce due primitive di dati: torch.utils.data.DataLoader e torch.utils.data.Dataset che ti consentono di utilizzare set di dati pre-caricati oltre ai tuoi dati personalizzati. 

<font color="green">Dataset:</font>
* l Dataset è una classe di PyTorch che memorizza i campioni di dati e le loro etichette corrispondenti.
* Si tratta essenzialmente di una rappresentazione dei dati grezzi senza alcuna elaborazione.
* Può essere personalizzato per adattarsi alle specifiche esigenze del tuo set di dati, implementando i metodi __getitem__() e __len__().
* Un esempio comune è la sottoclasse torch.utils.data.Dataset che definisce un set di dati come FashionMNIST o CIFAR-10.

<font color="green">Dataloader:</font>
* Il DataLoader è una classe di PyTorch che avvolge un Dataset e lo rende iterabile.
* Fornisce funzionalità aggiuntive come il batching dei dati, il campionamento casuale e il caricamento parallelo dei dati.
* Semplifica l'accesso ai campioni di dati durante l'addestramento del modello.
* Può essere configurato con parametri come batch_size, shuffle, num_workers, ecc., per personalizzare il comportamento del caricamento dei dati.
* In sostanza, il DataLoader "confeziona" i dati del Dataset in mini-batch gestibili per l'addestramento del modello.


Ecco un esempio di come caricare il set di dati Fashion-MNIST da TorchVision. ***Fashion-MNIST*** è un dataset di dati di immagini degli articoli di Zalando composto da 60.000 esempi di train e 10.000 esempi di test.
Ogni esempio comprende un'immagine in scala di grigi 28×28 e un'etichetta associata da una delle 10 classi.

In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
print(f"Training Data info: {training_data}")
print(f"Test Data info: {test_data}")


100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Training Data info: Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
Test Data info: Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


Nello specifico:

* ***Root:*** È il percorso principale dove sono memorizzati i dati.

* ***Train/Test:*** Indica se il set di dati specificato è per l'addestramento o per il test del modello.

* ***Download:*** Impostando questo parametro su True, i dati verranno scaricati automaticamente da Internet se non sono presenti nella radice specificata.

* ***Transform e target_transform:*** Specificano le trasformazioni da applicare alle caratteristiche (input) e alle etichette (output) dei dati. Questo è utile per eseguire operazioni come la normalizzazione dei dati o la conversione delle etichette in un formato desiderato prima di passarli al modello.


Possiamo indicizzare manualmente i set di dati come una lista: ***training_data[index]***. Utilizziamo matplotlib per visualizzare alcuni campioni nei nostri dati di addestramento.